# New Raster Mask

Created 2019-01-15
This program takes the code from Raster Mask and cleans it up for use to do two things:

1. Create new rasters with a simple mask applied
2. Complete connected components of two masks

Check crs of raster
'EPSG:4326' ... or 

In [1]:
import rasterio
from rasterio.mask import mask
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

### Inputs

In [2]:
temp_data = '../../temp_data/'
raster_interim = '../../data/interim/'
raster_raw = '../../data/'

### Mask Functions

In [3]:
# Let's try to make the mask directy with rasterio

def raster_mask_poly(raster, band, density, crs):
    """Fucntion makes numpy array for of raster then applies mask.
    requires rasterio object, band number, and density as float on int and crs of raster
    returns dict of polygons and numpy mask
    """

    mask = raster.read(band)
    mask[mask < density] = 0 # mask as zeros 
    mask[mask >= density] = 1 
    
    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(mask, transform=raster.transform):
        
        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326) removed 2019-01-15
        geom = rasterio.warp.transform_geom(raster.crs, crs, geom, precision=6) # WGS84 (EPSG 4326)
    
    # turn geom into Polygon object from shapely    
    # Polygon(geom['coordinates'][0])
    poly_gdf = gpd.GeoDataFrame()
    for i, poly in enumerate(geom['coordinates']):
        poly_gdf.loc[i,'geometry']= Polygon(poly)

    return mask, geom, poly_gdf

In [4]:
def raster_mask(raster, band, density):
    "Function returns raster mask"
    
    mask = raster.read(band)
    mask[mask < density] = 0 # mask as zeros and ones
    mask[mask >= density] = 1
    
    return mask

### Polygons for >1500 ppl per km2

In [15]:
file = rasterio.open(raster_raw+'GHS_POP_GPW42015_GLOBE_R2000A_54009_1k_v1_0_Clip.tif')
crs = file.crs

In [16]:
mask1500m, geom1500, poly_gdf1500 = raster_mask_poly(file, 1, 1500, crs)

In [17]:
print(poly_gdf1500.head(6))
print(len(poly_gdf1500))

                                            geometry
0  POLYGON ((-25.462978 37.445441, -25.462978 -34...
1  POLYGON ((19.19249 -34.420127, 19.201473 -34.4...
2  POLYGON ((9.849966 37.30171, 9.849966 37.29272...
3  POLYGON ((22.103045 -34.186564, 22.112028 -34....
4  POLYGON ((19.237406 -34.411144, 19.246389 -34....
5  POLYGON ((18.985876 -34.150631, 19.021809 -34....
37631


In [18]:
# drop first row, which is bounding box of Africa and the write 
poly_gdf1500 = poly_gdf1500.drop(poly_gdf1500.index[0], axis = 0)
poly_gdf1500.head(6)

,geometry
1,"POLYGON ((19.19249 -34.420127, 19.201473 -34.4..."
2,"POLYGON ((9.849966 37.30171, 9.849966 37.29272..."
3,"POLYGON ((22.103045 -34.186564, 22.112028 -34...."
4,"POLYGON ((19.237406 -34.411144, 19.246389 -34...."
5,"POLYGON ((18.985876 -34.150631, 19.021809 -34...."
6,"POLYGON ((19.19249 -34.222497, 19.201473 -34.2..."


In [19]:
# write out polygons 

poly_gdf1500.to_file(temp_data+'WPE_1KM_2016_Pop_Clip.shp', driver='ESRI Shapefile')